# Extraer info de notas y silencios: tokenizar por nota en vectores binarios #

In [1]:
import jams
import numpy as np

# Cargar el archivo JAMS
jam = jams.load('./data/annotation/03_Rock3-117-Bb_solo.jams')

# Buscar anotaciones de notas MIDI
notes = jam.search(namespace='note_midi')
chords = jam.search(namespace='chord')[0].data

# Obtener el valor de los BPM (tempo)
bpm = jam.search(namespace='tempo')[0].data[0].value
bps = bpm / 60
# Duración de negra en segundos
quater_duration = 1 / bps

# Extraer todas las notas
all_notes = [note for sublist in [list(note.data) for note in notes] for note in sublist]
all_chords = [chord for chord in chords]
# Ordenar las notas por tiempo
all_notes.sort(key=lambda x: x.time)
all_chords.sort(key=lambda x: x.time)
# Inicializar vectores de altura, duración y desfase
pitch_vecs = []
dur_vecs = []
offset_vecs = []
start_times = []
# Duraciones permitidas en segundos (de menor a mayor)
duraciones_permitidas_seg = {
    'fusa': quater_duration / 8,
    'tresillo_semicorchea': quater_duration / 6,
    'fusa_puntillo': 1.5 * quater_duration / 8,
    'semicorchea': quater_duration / 4,
    'tresillo_corchea': quater_duration / 3,
    'semicorchea_puntillo': 1.5 * quater_duration / 4,
    'corchea': quater_duration / 2,
    'tresillo': (2 / 3) * quater_duration,
    'corchea_puntillo': 1.5 * quater_duration / 2,
    'negra': quater_duration,
    'negra_puntillo': quater_duration * 1.5,
    'blanca': 2 * quater_duration,
    'blanca_puntillo': 2 * quater_duration * 1.5,
    'redonda': 4 * quater_duration,
}
diccionario_tonicas = { 
    'C': 0,
    'C#': 1,
    'D': 2,
    'D#': 3,
    'E': 4,
    'F': 5,
    'F#': 6,
    'G': 7,
    'G#': 8,
    'A': 9,
    'A#': 10,
    'B': 11
}
diccionario_acordes = {
    'maj': [0, 4, 7],
    'min': [0, 3, 7],
    '7': [0, 4, 7, 10],
    'maj7': [0, 4, 7, 11],
    'min7': [0, 3, 7, 10],
    'dim': [0, 3, 6],
    'dim7': [0, 3, 6, 9],
    'aug': [0, 4, 8],
    'aug7': [0, 4, 8, 10],
    'sus2': [0, 2, 7],
    'sus4': [0, 5, 7],
    '7sus4': [0, 5, 7, 10],
    '6': [0, 4, 7, 9],
    'm6': [0, 3, 7, 9],
    '9': [0, 4, 7, 10, 14],
    'maj9': [0, 4, 7, 11, 14],
    'min9': [0, 3, 7, 10, 14],
    '11': [0, 4, 7, 10, 14, 17],
    'maj11': [0, 4, 7, 11, 14, 17],
    'min11': [0, 3, 7, 10, 14, 17],
    '13': [0, 4, 7, 10, 14, 17, 21],
    'maj13': [0, 4, 7, 11, 14, 17, 21],
    'min13': [0, 3, 7, 10, 14, 17, 21],
    '7#9': [0, 4, 7, 10, 15],
    '7b9': [0, 4, 7, 10, 13],
    '7#5': [0, 4, 8, 10],
    '7b5': [0, 4, 6, 10],
    'hdim7': [0, 3, 6, 10],
    }

# Lista de nombres de duraciones
nombres_duraciones = list(duraciones_permitidas_seg.keys())
# Duración mínima permitida
duracion_minima = min(duraciones_permitidas_seg.values())
tresillo_minimo = quater_duration / 6

# Definir los desfases permitidos en segundos
max_offset = 4 * quater_duration - duracion_minima
offsets_permitidos_seg = [i * duracion_minima for i in range(int(max_offset // duracion_minima))]
# Añadir múltiplos de tresillos al diccionario de desfases permitidos
num_tresillos = int(max_offset / tresillo_minimo)
for i in range(num_tresillos):
    offset_tresillo = i * tresillo_minimo
    offsets_permitidos_seg.append(offset_tresillo)
# Ordenar los desfases permitidos
offsets_permitidos_seg.sort()
# Función para encontrar la duración permitida más cercana a una duración dada en segundos
def encontrar_duracion_permitida(duracion):
    duracion_permitida_cercana = min(duraciones_permitidas_seg.values(), key=lambda x: abs(x - duracion))
    return duracion_permitida_cercana
def encontrar_nombre_duracion(duracion):
    duracion_permitida_cercana = min(duraciones_permitidas_seg.values(), key=lambda x: abs(x - duracion))
    return list(duraciones_permitidas_seg.keys())[list(duraciones_permitidas_seg.values()).index(duracion_permitida_cercana)]
# Función para representar una duración como un vector binario
def duracion_a_vec(duracion):
    duracion_permitida_cercana = encontrar_duracion_permitida(duracion)
    vector_binario = np.zeros(len(duraciones_permitidas_seg))
    for i, duracion_permitida in enumerate(duraciones_permitidas_seg.values()):
        if duracion_permitida == duracion_permitida_cercana:
            vector_binario[i] = 1
    return vector_binario

# Función para representar un desfase como un vector binario
def offset_a_vec(offset):
    # Buscar el desfase permitido más cercano
    offset_permitido_cercano = min(offsets_permitidos_seg, key=lambda x:abs(x-offset))
    vector_binario = np.zeros(len(offsets_permitidos_seg))
    for i, offset_permitido in enumerate(offsets_permitidos_seg):
        if offset_permitido == offset_permitido_cercano:
            vector_binario[i] = 1
    return vector_binario
def acorde_a_vec(acorde):
    acorde_vec = np.zeros(12)
    tonica = acorde.value.split(':')[0]
    tipo_acorde = acorde.value.split(':')[1]
    idx_tonica = diccionario_tonicas[tonica]
    idx_tipo_acorde = diccionario_acordes[tipo_acorde]
    for idx in idx_tipo_acorde:
        acorde_vec[(idx_tonica + idx) % 12] = 1    
    return acorde_vec
# Función para dividir un silencio largo en duraciones permitidas
def dividir_silencio(delta_t):
    max_dur = 4 * quater_duration 
    duraciones_divididas = []
    while delta_t > max_dur:
        duracion_permitida = encontrar_duracion_permitida(delta_t)
        duraciones_divididas.append(duracion_permitida)
        delta_t -= duracion_permitida
    if delta_t <= max_dur:
        duracion_permitida = encontrar_duracion_permitida(delta_t)
        duraciones_divididas.append(duracion_permitida)
        
    return duraciones_divididas



# Inicializar delta_t con el tiempo de inicio de la primera nota y offset inicial
delta_t = all_notes[0].time
acorde_actual = all_chords[0]
tiempo_actual = 0
# Iterar sobre las notas ordenadas
for i in range(len(all_notes)):
    note = all_notes[i]
    pitch_vec = np.zeros(129)
    duration = note.duration
    start_time = note.time
    pitch_midi = note.value
    #encargarse de los silencios
    # Si delta_t es mayor o igual a la duración mínima permitida, agregar silencios divididos
    if delta_t >= duracion_minima:
        tiempo_actual = start_time-delta_t
        duraciones_silencio = dividir_silencio(delta_t)
        for duracion_silencio in duraciones_silencio:
            dur_vecs.append(duracion_a_vec(duracion_silencio))
            current_offset = tiempo_actual % (4 * quater_duration)
            offset_vecs.append(offset_a_vec(current_offset))
            start_times.append(tiempo_actual)
            tiempo_actual += duracion_silencio
            silencio_vec = np.zeros(129)
            silencio_vec[-1] = 1  # La última componente es 1 para representar el silencio
            pitch_vecs.append(silencio_vec)
    tiempo_actual = start_time
    start_times.append(tiempo_actual)
    # Redondear pitch
    midi_note = round(pitch_midi)
    pitch_vec[midi_note] = 1
    # Añadir vector de pitch a la lista
    pitch_vecs.append(pitch_vec)
    dur_vecs.append(duracion_a_vec(duration))
    offset_vecs.append(offset_a_vec(tiempo_actual % (4 * quater_duration)))

    # Actualizar delta_t y offset para la próxima iteración
    if i < len(all_notes) - 1:
        delta_t = all_notes[i + 1].time - (start_time + duration)
    # Encontrar el nombre de la duración permitida más cercana
    nombre_duracion = encontrar_nombre_duracion(duration)
    vec_offset = offset_a_vec(note.time%(4*quater_duration))
    idx_offset = np.where(vec_offset == 1)[0][0]
    #print(f"Tiempo: {start_time}, Desfase: {idx_offset}, Nota: {midi_note}, Duración nombre: {nombre_duracion}")
llaves_dur = list(duraciones_permitidas_seg.keys())
# Ejemplo de impresión de vectores
print(f'Altura {[np.where(pitch == 1)[0][0] for pitch in pitch_vecs]}')
print(f'Duración {[llaves_dur[np.where(dur == 1)[0][0]] for dur in dur_vecs]}')
#print(f'Duración {[np.where(dur == 1)[0][0] for dur in dur_vecs]}')
print(f'Desfase {[np.where(offset == 1)[0][0] for offset in offset_vecs]}')


Altura [128, 65, 67, 70, 67, 65, 67, 69, 72, 69, 128, 62, 65, 67, 65, 128, 62, 63, 65, 69, 65, 62, 57, 62, 128, 67, 70, 67, 63, 63, 61, 62, 70, 62, 70, 62, 70, 62, 128, 67, 70, 67, 67, 128, 63, 63, 128, 63, 69, 63, 69, 63, 128, 61, 62, 70, 65, 62, 67, 65, 62, 64, 128, 69, 65, 69, 128, 62, 65, 67, 65, 62, 60, 62, 63, 63, 69, 72, 65, 60, 128, 67, 70, 67, 63, 67, 128, 61, 128, 70, 62, 63, 72, 73, 65, 74, 128, 67, 70, 67, 63, 67, 128, 62, 63, 128, 69, 72, 69, 75]
Duración ['negra', 'fusa_puntillo', 'semicorchea', 'blanca', 'negra', 'corchea', 'semicorchea_puntillo', 'semicorchea_puntillo', 'tresillo_corchea', 'blanca', 'blanca', 'fusa_puntillo', 'tresillo_corchea', 'semicorchea_puntillo', 'tresillo', 'tresillo_corchea', 'semicorchea_puntillo', 'semicorchea_puntillo', 'corchea', 'semicorchea_puntillo', 'corchea', 'negra_puntillo', 'semicorchea_puntillo', 'semicorchea_puntillo', 'negra', 'corchea', 'corchea', 'blanca', 'negra', 'semicorchea_puntillo', 'semicorchea_puntillo', 'corchea', 'negr

In [2]:
print(len(pitch_vecs)+len(dur_vecs)+len(offset_vecs))

267


# Extraer info armónica: vectorizar acordes #
Agregar acordes iterando sobre start_times y verificando que start_times[i] esté entre chord_time y chord_time + chord_duration

In [3]:
chord_vecs = []
curr_chord_idx = 0
for i in range(len(start_times)):
    acorde_actual = all_chords[curr_chord_idx]
    print(acorde_actual)
    chord_time = acorde_actual.time
    chord_duration = acorde_actual.duration
    if start_times[i] > chord_time + chord_duration:
        curr_chord_idx += 1
        acorde_vec = acorde_a_vec(all_chords[curr_chord_idx])
        chord_vecs.append(acorde_vec)
    else:
        acorde_vec = acorde_a_vec(all_chords[curr_chord_idx])
        chord_vecs.append(acorde_vec)

print(f'Acordes {[np.where(chord == 1)[0] for chord in chord_vecs]}')
len(chord_vecs[0]), len(pitch_vecs[0]), len(dur_vecs[0]), len(offset_vecs[0]), len(start_times)


Observation(time=0.0, duration=2.051282051282051, value='A#:maj', confidence=None)
Observation(time=0.0, duration=2.051282051282051, value='A#:maj', confidence=None)
Observation(time=0.0, duration=2.051282051282051, value='A#:maj', confidence=None)
Observation(time=0.0, duration=2.051282051282051, value='A#:maj', confidence=None)
Observation(time=0.0, duration=2.051282051282051, value='A#:maj', confidence=None)
Observation(time=0.0, duration=2.051282051282051, value='A#:maj', confidence=None)
Observation(time=0.0, duration=2.051282051282051, value='A#:maj', confidence=None)
Observation(time=0.0, duration=2.051282051282051, value='A#:maj', confidence=None)
Observation(time=2.051282051282051, duration=2.051282051282051, value='F:maj', confidence=None)
Observation(time=2.051282051282051, duration=2.051282051282051, value='F:maj', confidence=None)
Observation(time=2.051282051282051, duration=2.051282051282051, value='F:maj', confidence=None)
Observation(time=2.051282051282051, duration=2.0

(12, 129, 14, 54, 110)